# tf.Tensor

In [1]:
import os
import numpy as np
import tensorflow as tf

print(f"tf version: {tf.__version__}")

def get_free_gpu():
    gpu_status = os.popen(
        'nvidia-smi --query-gpu=name,memory.free --format=csv').read().strip().split('\n')
    gpu_status.pop(0)
    gpu_mem = [int(stat.split(',')[-1].strip().split()[0])
               for stat in gpu_status]
    chosen_id = gpu_mem.index(max(gpu_mem))
    return chosen_id, gpu_mem[chosen_id]


def select_a_gpu():
    free_gpu_id, free_gpu_mem = get_free_gpu()
    os.environ['CUDA_VISIBLE_DEVICES'] = str(free_gpu_id)
    print(f"Selected GPU - {free_gpu_id}")

select_a_gpu()


tf version: 2.5.1
Selected GPU - 1


In [2]:
# Compute some values using a Tensor
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
d = tf.constant([[1.0, 1.0], [0.0, 1.0]])
e = tf.matmul(c, d)
print(e)


tf.Tensor(
[[1. 3.]
 [3. 7.]], shape=(2, 2), dtype=float32)


In [3]:
c

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

In [4]:
d

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [0., 1.]], dtype=float32)>

In [5]:
e


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 3.],
       [3., 7.]], dtype=float32)>

In [6]:
e.numpy()


array([[1., 3.],
       [3., 7.]], dtype=float32)

In [7]:
a = np.array([1, 2, 3])
b = tf.constant(a)
a[0] = 4

b


<tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 2, 3])>

In [8]:
a


array([4, 2, 3])

In [9]:
a.shape


(3,)

In [10]:
a = tf.constant([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0]
])
print(a.shape)


(2, 3)


In [11]:
inputs = tf.keras.Input(shape = [10])
# Unknown batch size
print(inputs.shape)


(None, 10)


In [12]:
@tf.function
def my_matmul(a, b):
  result = a @ b
  # the `print` executes during tracing.
  print("@@ Result shape: ", result.shape)
  return result

In [13]:
f = my_matmul.get_concrete_function(
  tf.TensorSpec([None,3]),
  tf.TensorSpec([3,5])
)


@@ Result shape:  (None, 5)


In [14]:
cf = my_matmul.get_concrete_function(
  tf.TensorSpec([None, 3]),
  tf.TensorSpec([4, 5])
)

ValueError: in user code:

    <ipython-input-12-97cef18a59dd>:3 my_matmul  *
        result = a @ b
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:1250 binary_op_wrapper
        raise e
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:1234 binary_op_wrapper
        return func(x, y, name=name)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3598 matmul_wrapper
        return matmul(a, b, name=name)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3490 matmul
        a, b, transpose_a=transpose_a, transpose_b=transpose_b, name=name)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gen_math_ops.py:5718 mat_mul
        name=name)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:750 _apply_op_h

In [ ]:
@tf.function
def my_fun(a):
  a = tf.ensure_shape(a, [5, 5])
  # the `print` executes during tracing.
  print("Result shape: ", a.shape)
  return a

In [ ]:
cf = my_fun.get_concrete_function(
  tf.TensorSpec([None, None]))


Result shape:  (5, 5)


# Ref

In [ ]:
x = tf.constant(5)
y = tf.constant(10)
z = tf.constant(10)
tensor_set = {x, y, z}

tensor_dict = {x: 'five', y: 'ten'}


TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.

In [ ]:
tensor_set = {x.ref(), y.ref(), z.ref()}

x.ref() in tensor_set

tensor_dict = {x.ref(): 'five', y.ref(): 'ten', z.ref(): 'ten'}

tensor_dict[y.ref()]

'ten'

In [ ]:
a = np.array([1, 2, 3])
b = tf.constant(a)
a[0] = 4
print(b)  # tf.Tensor([4 2 3], shape=(3,), dtype=int64)

tf.Tensor([1 2 3], shape=(3,), dtype=int64)


In [ ]:
a

array([4, 2, 3])

In [ ]:
b

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 2, 3])>

In [15]:
x = tf.constant(5)
x.ref()

<Reference wrapping <tf.Tensor: shape=(), dtype=int32, numpy=5>>

In [16]:
x.ref().deref()

<tf.Tensor: shape=(), dtype=int32, numpy=5>

In [19]:
t = tf.constant([[1,2,3]])
t.shape

TensorShape([1, 3])

In [22]:
t.set_shape([1, 3])
t.shape

TensorShape([1, 3])

In [23]:
t.set_shape([1,None])

In [24]:
t

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[1, 2, 3]], dtype=int32)>

In [26]:
t = tf.keras.Input(shape=[None, None, 3])
t.shape

TensorShape([None, None, None, 3])

In [27]:
t.set_shape([None, 224, 224, None])
print(t.shape)

(None, 224, 224, 3)


# Load Image

In [29]:
@tf.function
def load_image(filename):
  raw = tf.io.read_file(filename)
  image = tf.image.decode_png(raw, channels=3)
  # the `print` executes during tracing.
  print("Initial shape: ", image.shape)
  
  image.set_shape([28, 28, 3])

  print("Final shape: ", image.shape)
  
  return image

In [30]:
cf = load_image.get_concrete_function(
    tf.TensorSpec([], dtype=tf.string)
)

Initial shape:  (None, None, 3)
Final shape:  (28, 28, 3)


In [ ]:
concrete_parse = my_parse.get_concrete_function(
    tf.TensorSpec([], dtype=tf.string))